<h2>Kaggle Bike Sharing Demand Dataset Normalization</h2>
Normalize 'temp','atemp','humidity','windspeed' and store the train and test files

Bug fix - Using Normalizer for transformation

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import Normalizer
from sklearn.decomposition import PCA
# http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html

import boto3
import sagemaker.amazon.common as smac

In [2]:
columns = ['count', 'season', 'holiday', 'workingday', 'weather', 'temp',
       'atemp', 'humidity', 'windspeed', 'year', 'month', 'day', 'dayofweek','hour']

cols_normalize = ['temp','atemp','humidity','windspeed']

In [5]:
df = pd.read_csv('train.csv', parse_dates=['datetime'])
df_test = pd.read_csv('test.csv', parse_dates=['datetime'])

In [6]:
# We need to convert datetime to numeric for training.
# Let's extract key features into separate numeric columns
def add_features(df):
    df['year'] = df['datetime'].dt.year
    df['month'] = df['datetime'].dt.month
    df['day'] = df['datetime'].dt.day
    df['dayofweek'] = df['datetime'].dt.dayofweek
    df['hour'] = df['datetime'].dt.hour

In [7]:
add_features(df)
add_features(df_test)

In [8]:
df["count"] = df["count"].map(np.log1p)

In [9]:
df.head(2)

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count,year,month,day,dayofweek,hour
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,2.833213,2011,1,1,5,0
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,3.713572,2011,1,1,5,1


In [10]:
df_test.head(2)

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,year,month,day,dayofweek,hour
0,2011-01-20 00:00:00,1,0,1,1,10.66,11.365,56,26.0027,2011,1,20,3,0
1,2011-01-20 01:00:00,1,0,1,1,10.66,13.635,56,0.0000,2011,1,20,3,1


In [11]:
# Normalize the dataset
transformer = Normalizer()

In [12]:
# Normalization parameters based on Training
transformer.fit(df[cols_normalize])

Normalizer()

In [13]:
def transform_data(scaler, df, columns):
    transformed_data = scaler.transform(df[columns])
    df_transformed = pd.DataFrame(transformed_data, columns=columns)
    
    for col in df_transformed.columns:
        df[col] = df_transformed[col]

In [14]:
transform_data(transformer, df, cols_normalize)
transform_data(transformer, df_test, cols_normalize)

In [15]:
df.head(2)

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count,year,month,day,dayofweek,hour
0,2011-01-01 00:00:00,1,0,0,1,0.118761,0.173736,0.977605,0.0,3,13,2.833213,2011,1,1,5,0
1,2011-01-01 01:00:00,1,0,0,1,0.110467,0.166986,0.979751,0.0,8,32,3.713572,2011,1,1,5,1


In [16]:
df_test.head(2)

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,year,month,day,dayofweek,hour
0,2011-01-20 00:00:00,1,0,1,1,0.167404,0.178475,0.879419,0.408344,2011,1,20,3,0
1,2011-01-20 01:00:00,1,0,1,1,0.181869,0.232625,0.955410,0.000000,2011,1,20,3,1


In [17]:
# Store Original train and test data in normalized form
df.to_csv('train_normalized.csv',index=False, columns=columns)
df_test.to_csv('test_normalized.csv',index=False)

In [18]:
# Store only the 4 numeric colums for PCA Training and Test
# Data Needs to be normalized

In [19]:
def write_recordio_file (filename, x, y=None):
    with open(filename, 'wb') as f:
        smac.write_numpy_to_dense_tensor(f, x, y)

In [20]:
# Store All Normalized data as RecordIO File for PCA Training in SageMaker
# Need to pass as an array to create RecordIO file
X = df[['temp','atemp','humidity','windspeed']].values
write_recordio_file('bike_train_numeric_columns.recordio',X)